In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from copy import deepcopy
from scipy.stats import entropy
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)

In [5]:
# Insights
"""
1. Must of the data is imputed

2. There are multiple records per patient, and some are conflicting.
There are 2 type of conflicts reasons - due to different sample_type and due to different plate.
Most of the conflicts are because of sample_type.
The patients with conflicting plates are concetrated (100 conflicts in about 3 patients)

3. GP (genotype probability), There are conflicts in the classified GT and GP (0.05%).
Must of the conflicts are of IMPUTED variants. Must of the patients don't have conflicts, and those who has
have up to 0.1% of the variants.
The variants themselfs conflicts around 0.1% of the users, with ourlier that conflict 5%.

4. GD (genotype dosage) - ranges for 0 to 2, the amount of evidence of the genotype. Meaning, GD of 0 
would suggest GT of 0 and GD of 2 would suggest a GT of 2.

Other to be tested:
conflicts in labels - how to resolve..

"""

In [ ]:
# Actions:
# Filter patients: 'TCGA-AA-3531'

In [ ]:
# This is just a POC!!!
# Goal now - create a unified file for all patients and resolve the conflicts
# Important tasks:
# 1. Choose a single sample per patient
# 2. Remove outliers SNPs - optional

In [ ]:
# Experiments:
# 0. Resolve conflicts according to avg entropy. Analyzing the results.
# 1. Use only the tumor types, choosing the plate conflicts according to higher mean entropy.
# 2. The same as 1. but adding the non conflicting blood samples.
# 3. Using all, chossing samples according to highest mean entropy.
# 4. Analyze the results in term of problamtic variants. - Imputed/Typed, variants that are not predicted well, variants with low R2, ect.
# 5. Analyze the results in terms of patients?

# Loading and Transforming Data

In [2]:
df = pd.read_csv('VCF_and_Matched_RACE.2023-04-23.csv')
df.drop(columns=['TCGA-AA-3531-10A-01D-0819-01', 'TCGA-AA-3531-11A-01D-1550-01'], inplace=True)
df.head(3)

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,TCGA-AA-3693-10A-01D-0903-01,TCGA-D5-6930-10A-01D-1923-01,TCGA-G4-6628-10A-01D-1834-01,TCGA-CA-5254-10A-01D-1834-01,TCGA-CM-4746-10A-01D-1406-01,TCGA-AA-A010-10A-01D-A008-01,TCGA-AA-3517-10A-01D-0819-01,TCGA-AA-3666-10A-01D-0903-01,TCGA-AZ-6606-11A-01D-1834-01,TCGA-D5-6532-10A-01D-1717-01,TCGA-A6-2674-10A-01D-0819-01,TCGA-AA-3538-10A-01D-0819-01,TCGA-AA-3520-11A-01D-1549-01,TCGA-AZ-6601-11A-01D-1770-01,TCGA-A6-5661-10A-01D-1649-01,TCGA-AM-5820-10A-01D-1649-01,TCGA-A6-2685-10A-01D-1406-01,TCGA-AA-3866-10A-01D-1018-01,TCGA-AU-6004-10A-01D-1717-01,TCGA-G4-6588-10A-01D-1770-01,TCGA-AA-3518-11A-01D-1549-01,TCGA-D5-5538-10A-02D-1649-01,TCGA-SS-A7HO-10A-01D-A36Z-01,TCGA-AA-3516-10A-01D-0824-01,TCGA-AA-A029-10A-01D-A008-01,TCGA-AZ-4323-10A-01D-1834-01,TCGA-G4-6295-10A-01D-1717-01,TCGA-AA-A01I-10A-01D-A008-01,TCGA-AZ-4313-10A-01D-1406-01,TCGA-AA-A01G-10A-01D-A003-01,TCGA-A6-2675-11A-01D-1717-01,TCGA-AA-3518-10A-01D-0824-01,TCGA-CM-6171-10A-01D-1649-01,TCGA-AA-3872-10A-01D-1018-01,TCGA-AA-3858-10A-01D-0903-01,TCGA-A6-2684-10A-01D-A274-01,TCGA-AA-3831-10A-01D-0903-01,TCGA-AZ-5407-10A-01D-1717-01,TCGA-A6-2672-10A-01D-0824-01,TCGA-A6-6137-10A-01D-1770-01,TCGA-AA-3679-10A-01D-0903-01,TCGA-A6-6142-10A-01D-1770-01,TCGA-D5-6898-10A-01D-1923-01,TCGA-AA-3521-11A-01D-1549-01,TCGA-AZ-4614-10A-01D-1406-01,TCGA-T9-A92H-10A-01D-A36Z-01,TCGA-AA-3514-11A-01D-1549-01,TCGA-AA-3966-10A-01D-1427-01,TCGA-AA-3519-11A-01D-1549-01,TCGA-DM-A285-10A-01D-A16U-01,TCGA-A6-5665-10A-01D-1649-01,TCGA-AA-3524-10A-01D-0819-01,TCGA-A6-3809-10A-01D-1450-01,TCGA-AA-A02O-11A-11D-A080-01,TCGA-AA-3715-10A-01D-0903-01,TCGA-AA-3812-10A-01D-0903-01,TCGA-A6-2671-10A-01D-1406-01,TCGA-D5-5540-10A-01D-1649-01,TCGA-CM-6675-10A-01D-1834-01,TCGA-AZ-6608-11A-01D-1834-01,TCGA-G4-6625-11A-01D-1770-01,TCGA-DM-A28F-10A-01D-A16U-01,TCGA-D5-6536-10A-01D-1717-01,TCGA-A6-2676-10A-01D-0824-01,TCGA-AA-3841-10A-01D-0903-01,TCGA-DM-A288-10A-01D-A16U-01,TCGA-AZ-4681-10A-01D-1406-01,TCGA-CM-6168-10A-01D-1649-01,TCGA-AA-3970-10A-01D-1018-01,TCGA-CM-6163-10A-01D-1649-01,TCGA-DM-A0X9-10A-01D-A151-01,TCGA-AA-3856-10A-01D-0903-01,TCGA-DM-A280-10A-01D-A16U-01,TCGA-DM-A0XF-10A-01D-A151-01,TCGA-A6-6782-10A-01D-1834-01,TCGA-AA-3986-10A-01D-1018-01,TCGA-A6-2683-11A-01D-1549-01,TCGA-F4-6569-10A-01D-1770-01,TCGA-A6-5664-10A-01D-1834-01,TCGA-QG-A5YV-10A-01D-A28F-01,TCGA-G4-6303-10A-01D-1770-01,TCGA-CK-6746-10A-01D-1834-01,TCGA-AA-3952-10A-01D-1018-01,TCGA-CA-6718-10A-01D-1834-01,TCGA-A6-2682-11A-01D-1549-01,TCGA-AA-3845-10A-01D-1018-01,TCGA-A6-6138-10A-01D-1770-01,TCGA-AA-3548-10A-01D-0819-01,TCGA-D5-6539-10A-01D-1717-01,TCGA-AA-3502-11A-01D-1406-01,TCGA-5M-AATA-10A-01D-A40O-01,TCGA-AZ-5403-10A-01D-1649-01,TCGA-A6-2675-10A-01D-1717-01,TCGA-CK-5915-10A-01D-1649-01,TCGA-AD-6890-10A-01D-1923-01,TCGA-AA-3655-11A-01D-1717-01,TCGA-NH-A8F8-10A-01D-A40O-01,TCGA-AA-3939-10A-01D-1018-01,TCGA-CM-5348-10A-01D-1717-01,TCGA-AA-A02J-10A-01D-A008-01,TCGA-D5-6920-10A-01D-1923-01,TCGA-A6-3808-10A-01D-1450-01,TCGA-A6-5656-10A-01D-A274-01,TCGA-A6-5657-10A-01D-1649-01,TCGA-A6-4105-10A-01D-1770-01,TCGA-A6-6781-10A-01D-1923-01,TCGA-DM-A1D0-10A-01D-A151-01,TCGA-CA-6719-10A-01D-1834-01,TCGA-AY-4071-10A-01D-1427-01,TCGA-AA-A00W-10A-01D-A003-01,TCGA-G4-6626-10A-01D-1770-01,TCGA-AD-6963-10A-01D-1923-01,TCGA-AA-A02Y-10A-01D-A080-01,TCGA-AA-A004-10A-01D-A008-01,TCGA-AA-A00U-10A-01D-A003-01,TCGA-AA-3833-10A-01D-0903-01,TCGA-G4-6320-11A-01D-1717-01,TCGA-AA-3815-10A-01D-1018-01,TCGA-CM-6172-10A-01D-1649-01,TCGA-AA-3688-10A-01D-0903-01,TCGA-CK-4951-10A-01D-1406-01,TCGA-A6-5665-11A-01D-1649-01,TCGA-A6-5661-11A-01D-1649-01,TCGA-A6-A565-10A-01D-A28F-01,TCGA-AA-A03J-11A-11D-A080-01,TCGA-F4-6809-10A-01D-1834-01,TCGA-AA-A01F-10A-01D-A003-01,TCGA-AA-3534-11A-01D-1550-01,TCGA-D5-5539-10A-01D-1649-01,TCGA-AA-3848-10A-01D-0903-01,TCGA-AD-A5EK-10A-01D-A28F-01,TCGA-AA-3526-10A-01D-0819-01,TCGA-CM-5862-10A-01D-1649-01,TCGA-CM-6169-10A-01D-1649-01,TCGA-AA-3949-10A-01D-1018-01,TCGA-AZ-6600-11A-01D-1770-01,TCGA-CM-6

In [3]:
records = []
for i, row in df.iterrows():
    info = row['INFO']
    info_split = info.split(';')
    is_imputed = info_split[-1]
    args = info_split[:-1]
    dict_row = {'IS_IMPUTED': is_imputed}
    for arg in args:
        key, val = arg.split('=')
        val = float(val)
        dict_row[key] = val
    records.append(dict_row)
df = pd.concat([pd.DataFrame.from_records(records), df], axis=1)
df.head(3)

,IS_IMPUTED,AF,MAF,R2,ER2,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,TCGA-AA-3693-10A-01D-0903-01,TCGA-D5-6930-10A-01D-1923-01,TCGA-G4-6628-10A-01D-1834-01,TCGA-CA-5254-10A-01D-1834-01,TCGA-CM-4746-10A-01D-1406-01,TCGA-AA-A010-10A-01D-A008-01,TCGA-AA-3517-10A-01D-0819-01,TCGA-AA-3666-10A-01D-0903-01,TCGA-AZ-6606-11A-01D-1834-01,TCGA-D5-6532-10A-01D-1717-01,TCGA-A6-2674-10A-01D-0819-01,TCGA-AA-3538-10A-01D-0819-01,TCGA-AA-3520-11A-01D-1549-01,TCGA-AZ-6601-11A-01D-1770-01,TCGA-A6-5661-10A-01D-1649-01,TCGA-AM-5820-10A-01D-1649-01,TCGA-A6-2685-10A-01D-1406-01,TCGA-AA-3866-10A-01D-1018-01,TCGA-AU-6004-10A-01D-1717-01,TCGA-G4-6588-10A-01D-1770-01,TCGA-AA-3518-11A-01D-1549-01,TCGA-D5-5538-10A-02D-1649-01,TCGA-SS-A7HO-10A-01D-A36Z-01,TCGA-AA-3516-10A-01D-0824-01,TCGA-AA-A029-10A-01D-A008-01,TCGA-AZ-4323-10A-01D-1834-01,TCGA-G4-6295-10A-01D-1717-01,TCGA-AA-A01I-10A-01D-A008-01,TCGA-AZ-4313-10A-01D-1406-01,TCGA-AA-A01G-10A-01D-A003-01,TCGA-A6-2675-11A-01D-1717-01,TCGA-AA-3518-10A-01D-0824-01,TCGA-CM-6171-10A-01D-1649-01,TCGA-AA-3872-10A-01D-1018-01,TCGA-AA-3858-10A-01D-0903-01,TCGA-A6-2684-10A-01D-A274-01,TCGA-AA-3831-10A-01D-0903-01,TCGA-AZ-5407-10A-01D-1717-01,TCGA-A6-2672-10A-01D-0824-01,TCGA-A6-6137-10A-01D-1770-01,TCGA-AA-3679-10A-01D-0903-01,TCGA-A6-6142-10A-01D-1770-01,TCGA-D5-6898-10A-01D-1923-01,TCGA-AA-3521-11A-01D-1549-01,TCGA-AZ-4614-10A-01D-1406-01,TCGA-T9-A92H-10A-01D-A36Z-01,TCGA-AA-3514-11A-01D-1549-01,TCGA-AA-3966-10A-01D-1427-01,TCGA-AA-3519-11A-01D-1549-01,TCGA-DM-A285-10A-01D-A16U-01,TCGA-A6-5665-10A-01D-1649-01,TCGA-AA-3524-10A-01D-0819-01,TCGA-A6-3809-10A-01D-1450-01,TCGA-AA-A02O-11A-11D-A080-01,TCGA-AA-3715-10A-01D-0903-01,TCGA-AA-3812-10A-01D-0903-01,TCGA-A6-2671-10A-01D-1406-01,TCGA-D5-5540-10A-01D-1649-01,TCGA-CM-6675-10A-01D-1834-01,TCGA-AZ-6608-11A-01D-1834-01,TCGA-G4-6625-11A-01D-1770-01,TCGA-DM-A28F-10A-01D-A16U-01,TCGA-D5-6536-10A-01D-1717-01,TCGA-A6-2676-10A-01D-0824-01,TCGA-AA-3841-10A-01D-0903-01,TCGA-DM-A288-10A-01D-A16U-01,TCGA-AZ-4681-10A-01D-1406-01,TCGA-CM-6168-10A-01D-1649-01,TCGA-AA-3970-10A-01D-1018-01,TCGA-CM-6163-10A-01D-1649-01,TCGA-DM-A0X9-10A-01D-A151-01,TCGA-AA-3856-10A-01D-0903-01,TCGA-DM-A280-10A-01D-A16U-01,TCGA-DM-A0XF-10A-01D-A151-01,TCGA-A6-6782-10A-01D-1834-01,TCGA-AA-3986-10A-01D-1018-01,TCGA-A6-2683-11A-01D-1549-01,TCGA-F4-6569-10A-01D-1770-01,TCGA-A6-5664-10A-01D-1834-01,TCGA-QG-A5YV-10A-01D-A28F-01,TCGA-G4-6303-10A-01D-1770-01,TCGA-CK-6746-10A-01D-1834-01,TCGA-AA-3952-10A-01D-1018-01,TCGA-CA-6718-10A-01D-1834-01,TCGA-A6-2682-11A-01D-1549-01,TCGA-AA-3845-10A-01D-1018-01,TCGA-A6-6138-10A-01D-1770-01,TCGA-AA-3548-10A-01D-0819-01,TCGA-D5-6539-10A-01D-1717-01,TCGA-AA-3502-11A-01D-1406-01,TCGA-5M-AATA-10A-01D-A40O-01,TCGA-AZ-5403-10A-01D-1649-01,TCGA-A6-2675-10A-01D-1717-01,TCGA-CK-5915-10A-01D-1649-01,TCGA-AD-6890-10A-01D-1923-01,TCGA-AA-3655-11A-01D-1717-01,TCGA-NH-A8F8-10A-01D-A40O-01,TCGA-AA-3939-10A-01D-1018-01,TCGA-CM-5348-10A-01D-1717-01,TCGA-AA-A02J-10A-01D-A008-01,TCGA-D5-6920-10A-01D-1923-01,TCGA-A6-3808-10A-01D-1450-01,TCGA-A6-5656-10A-01D-A274-01,TCGA-A6-5657-10A-01D-1649-01,TCGA-A6-4105-10A-01D-1770-01,TCGA-A6-6781-10A-01D-1923-01,TCGA-DM-A1D0-10A-01D-A151-01,TCGA-CA-6719-10A-01D-1834-01,TCGA-AY-4071-10A-01D-1427-01,TCGA-AA-A00W-10A-01D-A003-01,TCGA-G4-6626-10A-01D-1770-01,TCGA-AD-6963-10A-01D-1923-01,TCGA-AA-A02Y-10A-01D-A080-01,TCGA-AA-A004-10A-01D-A008-01,TCGA-AA-A00U-10A-01D-A003-01,TCGA-AA-3833-10A-01D-0903-01,TCGA-G4-6320-11A-01D-1717-01,TCGA-AA-3815-10A-01D-1018-01,TCGA-CM-6172-10A-01D-1649-01,TCGA-AA-3688-10A-01D-0903-01,TCGA-CK-4951-10A-01D-1406-01,TCGA-A6-5665-11A-01D-1649-01,TCGA-A6-5661-11A-01D-1649-01,TCGA-A6-A565-10A-01D-A28F-01,TCGA-AA-A03J-11A-11D-A080-01,TCGA-F4-6809-10A-01D-1834-01,TCGA-AA-A01F-10A-01D-A003-01,TCGA-AA-3534-11A-01D-1550-01,TCGA-D5-5539-10A-01D-1649-01,TCGA-AA-3848-10A-01D-0903-01,TCGA-AD-A5EK-10A-01D-A28F-01,TCGA-AA-3526-10A-01D-0819-01,TCGA-CM-5862-10A-01D-1649-01,TCGA-CM-6169-10A-01D-1649-01,TCGA-AA-3949-10A-01D-1018-01,TCGA-AZ-6600-

In [4]:
attr_cols = pd.Series(df.columns[~df.columns.str.startswith('TCGA')])
sample_ids = pd.Series(df.columns[df.columns.str.startswith('TCGA')])
df_t = pd.melt(df, id_vars=attr_cols, value_vars=sample_ids)
df_t.rename(columns={'variable': 'sample_id', 'value': 'gt_scores'}, inplace=True)
display(df_t.head(3))
display(df_t.shape)

,IS_IMPUTED,AF,MAF,R2,ER2,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,sample_id,gt_scores
0,TYPED,0.08269,0.08269,0.98341,0.74366,1,14665131,1:14665131:C:T,C,T,.,PASS,AF=0.08269;MAF=0.08269;R2=0.98341;ER2=0.74366;...,GT:DS:HDS:GP,TCGA-AA-3693-10A-01D-0903-01,"1|1:2:1,1:0,0,1"
1,IMPUTED,0.09059,0.09059,0.96957,NaN,1,18130969,1:18130969:G:C,G,C,.,PASS,AF=0.09059;MAF=0.09059;R2=0.96957;IMPUTED,GT:DS:HDS:GP,TCGA-AA-3693-10A-01D-0903-01,"1|0:0.997:0.997,0:0.003,0.997,0"
2,IMPUTED,0.07441,0.07441,0.95783,NaN,1,22587728,1:22587728:T:C,T,C,.,PASS,AF=0.07441;MAF=0.07441;R2=0.95783;IMPUTED,GT:DS:HDS:GP,TCGA-AA-3693-10A-01D-0903-01,"0|0:0:0,0:1,0,0"


(304200, 16)

In [5]:
df_t['patient_id'] = df_t['sample_id'].apply(lambda e: e[:12])
df_t['GT'] = df_t.gt_scores.apply(lambda elem: int(elem[0]) + int(elem[2]))
df_t['GP'] = df_t.gt_scores.apply(lambda elem: elem.split(':')[-1])
df_t['GP'] = df_t['GP'].apply(lambda elem: eval(f'[{elem}]'))
df_t['GD'] = df_t.gt_scores.apply(lambda elem: float(elem.split(':')[1]))
df_t['HDS'] = df_t.gt_scores.apply(lambda elem: elem.split(':')[2])
df_t['HDS'] = df_t['HDS'].apply(lambda elem: eval(f'[{elem}]'))
df_t.head(1)

,IS_IMPUTED,AF,MAF,R2,ER2,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,sample_id,gt_scores,patient_id,GT,GP,GD,HDS
0,TYPED,0.08269,0.08269,0.98341,0.74366,1,14665131,1:14665131:C:T,C,T,.,PASS,AF=0.08269;MAF=0.08269;R2=0.98341;ER2=0.74366;...,GT:DS:HDS:GP,TCGA-AA-3693-10A-01D-0903-01,"1|1:2:1,1:0,0,1",TCGA-AA-3693,2,"[0, 0, 1]",2.0,"[1, 1]"


In [6]:
# for i in range(4):
#     print(df_t.sample_id.apply(lambda e:e.split('-')[3+i]).unique())

df_t['sample_type'] = df_t.sample_id.apply(lambda e:e.split('-')[3][:2])
df_t.head(1)

,IS_IMPUTED,AF,MAF,R2,ER2,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,sample_id,gt_scores,patient_id,GT,GP,GD,HDS,sample_type
0,TYPED,0.08269,0.08269,0.98341,0.74366,1,14665131,1:14665131:C:T,C,T,.,PASS,AF=0.08269;MAF=0.08269;R2=0.98341;ER2=0.74366;...,GT:DS:HDS:GP,TCGA-AA-3693-10A-01D-0903-01,"1|1:2:1,1:0,0,1",TCGA-AA-3693,2,"[0, 0, 1]",2.0,"[1, 1]",10


In [7]:
df_t.sample_id.nunique(), df_t.patient_id.nunique()

(520, 459)

# EDA

## Imputation

In [15]:
df_t.groupby('ID').IS_IMPUTED.max().value_counts()

IMPUTED    427
TYPED      158
Name: IS_IMPUTED, dtype: int64

## Argmax Conflicts

In [22]:
df_argmax_conflicts = df_t[df_t.GT!=df_t.GP.apply(lambda a: a.index(max(a)))]
df_argmax_conflicts.head(1)

,IS_IMPUTED,AF,MAF,R2,ER2,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,sample_id,gt_scores,patient_id,GT,GP,GD,HDS
6386,IMPUTED,0.45261,0.45261,0.6059,NaN,19,55174213,19:55174213:C:T,C,T,.,PASS,AF=0.45261;MAF=0.45261;R2=0.6059;IMPUTED,GT:DS:HDS:GP,TCGA-A6-2674-10A-01D-0819-01,"0|0:0.941:0.465,0.476:0.28,0.498,0.221",TCGA-A6-2674,0,"[0.28, 0.498, 0.221]",0.941,"[0.465, 0.476]"


In [94]:
df_argmax_conflicts.IS_IMPUTED.value_counts()

IMPUTED    141
TYPED        1
Name: IS_IMPUTED, dtype: int64

In [32]:
df_argmax_conflicts.sample_id.value_counts().value_counts()

1    85
2    22
5     2
3     1
Name: sample_id, dtype: int64

In [28]:
df_argmax_conflicts.groupby('ID').sample_id.count().sort_values(ascending=False)[:5]

ID
19:55174213:C:T    27
12:51045449:A:G    12
14:54419106:A:C     4
13:31698294:G:A     4
5:90576279:G:A      4
Name: sample_id, dtype: int64

## GT Conflicts

In [261]:
df_conflicts_samples =  df_t.groupby(['patient_id', 'ID'], as_index=False).GT.nunique().query("GT>1").rename(columns={
    'GT': 'GT_NUM_CONFLICTS'
})

In [120]:
df_conflicts_samples.GT_NUM_CONFLICTS.value_counts()

2    740
Name: GT_NUM_CONFLICTS, dtype: int64

In [121]:
df_t.groupby('patient_id').sample_id.nunique().value_counts()

1    403
2     52
3      5
Name: sample_id, dtype: int64

In [122]:
df_gt_conflicts = df_t.merge(df_conflicts_samples, how='inner', on=['patient_id', 'ID'])
display(df_gt_conflicts.head(2))
display(df_gt_conflicts.shape)

,IS_IMPUTED,AF,MAF,R2,ER2,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,sample_id,gt_scores,patient_id,GT,GP,GD,HDS,sample_type,GT_NUM_CONFLICTS
0,IMPUTED,0.36902,0.36902,0.92177,NaN,11,7400790,11:7400790:G:A,G,A,.,PASS,AF=0.36902;MAF=0.36902;R2=0.92177;IMPUTED,GT:DS:HDS:GP,TCGA-AA-3517-10A-01D-0819-01,"0|0:0.262:0.262,0:0.738,0.262,0",TCGA-AA-3517,0,"[0.738, 0.262, 0]",0.262,"[0.262, 0]",10,2
1,IMPUTED,0.36902,0.36902,0.92177,NaN,11,7400790,11:7400790:G:A,G,A,.,PASS,AF=0.36902;MAF=0.36902;R2=0.92177;IMPUTED,GT:DS:HDS:GP,TCGA-AA-3517-11A-01D-1549-01,"0|1:0.703:0,0.703:0.297,0.703,0",TCGA-AA-3517,1,"[0.297, 0.703, 0]",0.703,"[0, 0.703]",11,2


(1652, 23)

In [96]:
df_gt_conflicts.IS_IMPUTED.value_counts()//2

IMPUTED    647
TYPED      179
Name: IS_IMPUTED, dtype: int64

In [68]:
(df_gt_conflicts.patient_id.value_counts()/(2*585))*100

TCGA-AA-3531    47.008547
TCGA-A6-5659    11.282051
TCGA-A6-3809    10.769231
TCGA-A6-2677    10.000000
TCGA-A6-3810     9.230769
TCGA-A6-6650     7.350427
TCGA-A6-6780     6.153846
TCGA-A6-2685     4.444444
TCGA-A6-5661     3.931624
TCGA-A6-2684     2.820513
TCGA-A6-5656     2.393162
TCGA-A6-5660     2.393162
TCGA-A6-3808     2.051282
TCGA-A6-2675     2.051282
TCGA-AA-3534     1.538462
TCGA-A6-5662     1.367521
TCGA-AA-3530     1.196581
TCGA-A6-6781     1.196581
TCGA-A6-3807     1.196581
TCGA-A6-2683     1.196581
TCGA-AA-3521     1.196581
TCGA-AA-3532     0.854701
TCGA-A6-5657     0.854701
TCGA-A6-4107     0.854701
TCGA-A6-2680     0.854701
TCGA-AA-3526     0.683761
TCGA-A6-2672     0.683761
TCGA-G4-6314     0.683761
TCGA-AA-3517     0.683761
TCGA-A6-2686     0.512821
TCGA-A6-2674     0.512821
TCGA-G4-6295     0.512821
TCGA-A6-5665     0.341880
TCGA-AA-3527     0.341880
TCGA-AA-3525     0.341880
TCGA-G4-6311     0.341880
TCGA-A6-2679     0.170940
TCGA-A6-2681     0.170940
TCGA-A6-2678

In [139]:
# REMOVING patient TCGA-AA-3531
df_t = df_t[df_t.patient_id!='TCGA-AA-3531']
df_gt_conflicts = df_gt_conflicts[df_gt_conflicts.patient_id!='TCGA-AA-3531']

In [140]:
# most of the conflicts are due to different in sample_type
df_gt_conflicts.groupby(['patient_id', 'ID']).sample_type.nunique().value_counts()

2    365
1    100
Name: sample_type, dtype: int64

In [141]:
# per patient see how the conflicts are distributed - sample_type or other
df_gt_conflicts.groupby(['patient_id', 'ID'], 
                        as_index=False).sample_type.nunique().groupby('patient_id').agg({
    'sample_type': [lambda s: (s==1).sum(), lambda s: (s==2).sum()],
})

sample_type           
              <lambda_0> <lambda_1>
patient_id                         
TCGA-A6-2672           0          4
TCGA-A6-2674           0          3
TCGA-A6-2675           0         12
TCGA-A6-2677           0         39
TCGA-A6-2678           0          1
TCGA-A6-2679           0          1
TCGA-A6-2680           0          5
TCGA-A6-2681           0          1
TCGA-A6-2683           0          7
TCGA-A6-2684           0         11
TCGA-A6-2685           0         26
TCGA-A6-2686           0          3
TCGA-A6-3807           0          7
TCGA-A6-3808           0         12
TCGA-A6-3809           0         42
TCGA-A6-3810           0         36
TCGA-A6-4107           0          5
TCGA-A6-5656          14          0
TCGA-A6-5657           0          5
TCGA-A6-5659           0         44
TCGA-A6-5660           0         14
TCGA-A6-5661           0         23
TCGA-A6-5662           0          8
TCGA-A6-5665           0          2
TCGA-A6-5666           0          1
TCGA-A6-5667           0          1
TCGA-A6-6650          43          0
TCGA-A6-6780          36          0
TCGA-A6-6781           7          0
TCGA-AA-3517           0          4
TCGA-AA-3519           0          1
TCGA-AA-3521           0          7
TCGA-AA-3524           0          1
TCGA-AA-3525           0          2
TCGA-AA-3526           0          4
TCGA-AA-3527           0          2
TCGA-AA-3530           0          7
TCGA-AA-3532           0          5
TCGA-AA-3534           0          9
TCGA-G4-6295           0          3
TCGA-G4-6311           0          2
TCGA-G4-6314           0          4
TCGA-G4-6322           0          1

In [142]:
df_gt_conflicts.groupby(['patient_id', 'ID']).GT.apply(lambda s: s.min()+s.max()).value_counts()

1    280
3    181
2      4
Name: GT, dtype: int64

In [143]:
df_gt_conflicts.groupby(['patient_id', 'ID'], as_index=False).GT.apply(lambda s: s.min()+s.max()).groupby(['GT']).agg({
    'ID': 'count',
    'patient_id': 'nunique'
})

,ID,patient_id
GT,,
1,280,39
2,4,3
3,181,35


In [160]:
# choosinging in liklyhood of conflicts, which are better?
# would be interesting to see also in the non-conflicts
df_gt_conflicts.GP.apply(lambda e: entropy(e)).round(1).value_counts().sort_index()

0.0    345
0.1    140
0.2     83
0.3     64
0.4     91
0.5     65
0.6     97
0.7    142
0.8     31
0.9     22
1.0     22
Name: GP, dtype: int64

In [175]:
df_gt_conflicts['GP_entropy'] = df_gt_conflicts.GP.apply(lambda e: entropy(e))
df_gt_conflicts['GP_max'] = df_gt_conflicts.GP.apply(lambda e: max(e))

In [177]:
df_gt_conflicts.groupby(['patient_id', 'ID']).agg({
    'GP_max': ['min', 'max', lambda s: abs(s.iloc[0]-s.iloc[1])]
})

GP_max                  
                                 min    max <lambda_0>
patient_id   ID                                       
TCGA-A6-2672 11:10286755:A:C   0.580  0.857      0.277
             9:131923949:C:T   0.807  1.000      0.193
             9:131930494:T:C   0.976  1.000      0.024
             9:131938913:T:C   0.931  1.000      0.069
TCGA-A6-2674 12:131045130:G:A  0.997  0.999      0.002
             5:1296486:A:G     0.524  0.999      0.475
             5:45265768:T:C    0.517  0.522      0.005
TCGA-A6-2675 10:114724473:A:G  0.515  0.854      0.339
             11:26012193:G:A   0.547  1.000      0.453
             12:30975469:C:A   0.526  0.999      0.473
             12:51045449:A:G   0.543  0.608      0.065
             17:10707241:G:A   0.541  0.943      0.402
             18:9701782:C:T    0.680  0.767      0.087
             19:41860296:A:G   0.792  0.942      0.150
             3:59930672:A:G    0.685  0.999      0.314
             3:66442435:C:G    0.972  0.997      0.025
             6:130925767:C:T   0.528  0.550      0.022
             6:69521978:G:A    0.976  1.000      0.024
             9:131930494:T:C   0.683  1.000      0.317
TCGA-A6-2677 10:8089136:T:G    0.956  1.000      0.044
             10:837407:A:G     0.995  1.000      0.005
             10:90580330:G:A   0.993  1.000      0.007
             11:127645687:A:G  0.991  1.000      0.009
             11:133112763:C:T  0.809  1.000      0.191
             11:61619829:C:A   0.514  0.969      0.455
             11:74409077:T:C   0.979  0.986      0.007
             12:111884608:T:C  0.574  0.996      0.422
             12:111973358:A:G  0.736  0.995      0.259
             12:131045130:G:A  0.692  0.822      0.130
             12:79908743:A:T   0.894  0.961      0.067
             13:111074915:C:T  0.930  0.999      0.069
             14:54445157:G:A   0.728  1.000      0.271
             15:28365618:A:G   0.993  0.996      0.000
             15:28410491:C:T   0.870  0.917      0.047
             15:33156386:G:A   0.815  1.000      0.185
             15:68177162:G:A   0.896  1.000      0.104
             19:53030198:C:G   0.682  0.968      0.011
             19:55174213:C:T   0.716  0.897      0.177
             1:222049820:G:T   0.716  0.988      0.272
             1:240081747:A:C   0.985  1.000      0.015
             1:62673037:T:C    0.942  1.000      0.058
             20:16189263:T:C   0.957  1.000      0.043
             20:57475191:A:G   0.968  1.000      0.032
             20:6690101:C:T    1.000  1.000      0.000
             20:6693128:C:T    0.990  1.000      0.010
             20:8449374:A:C    0.725  1.000      0.275
             3:175647165:G:A   0.541  1.000      0.459
             3:53088285:T:G    0.537  1.000      0.463
             3:94108663:G:A    0.688  0.992      0.304
             4:145621328:G:A   0.943  0.982      0.039
             4:145659064:T:C   0.991  0.999      0.008
             4:82693536:A:G    0.509  0.890      0.381
             5:134440426:G:A   0.984  1.000      0.016
             5:90576279:G:A    0.509  0.914      0.373
             6:54730344:A:G    0.993  0.996      0.003
             6:69521978:G:A    0.917  1.000      0.083
             7:75371146:T:C    0.873  0.883      0.008
             9:3761909:G:A     0.906  1.000      0.094
TCGA-A6-2678 12:51045449:A:G   0.524  0.549      0.025
TCGA-A6-2679 5:90576279:G:A    0.623  0.814      0.191
TCGA-A6-2680 13:105359644:A:G  0.746  0.966      0.220
             20:6404281:C:A    0.704  0.837      0.133
             5:1240204:C:T     0.544  0.865      0.321
             5:1240998:T:C     0.547  0.864      0.317
             6:162219854:C:T   0.903  0.963      0.060
TCGA-A6-2681 12:111884608:T:C  0.483  0.909      0.426
TCGA-A6-2683 11:100430053:A:C  0.540  0.855      0.315
             12:51045449:A:G   0.557  0.569      0.012
             12:6412036:T:G    0.976  0.993      0.017
             16:86252544:G:A   0.929  1.000      0.071
             19

In [182]:
df_gt_conflicts.groupby(['patient_id', 'ID'], as_index=False).GP_max.apply(lambda s: abs(s.iloc[0]-s.iloc[1])).groupby('ID').GP_max.mean().round(1).value_counts().sort_index()

0.0    65
0.1    77
0.2    55
0.3    26
0.4    18
0.5    10
Name: GP_max, dtype: int64

In [190]:
plate_diff_patients = df_gt_conflicts.groupby('patient_id',as_index=False).sample_type.nunique().query('sample_type==1').patient_id.values
len(plate_diff_patients)

4

In [219]:
df_gt_conflicts[df_gt_conflicts.patient_id.isin(plate_diff_patients)].groupby(['patient_id', 'ID'], as_index=False).GP_max.apply(lambda s: abs(s.iloc[0]-s.iloc[1])).groupby('ID').GP_max.mean()#.round(1).value_counts().sort_index()

ID
10:33405575:T:C     0.22000
10:52648454:C:T     0.50800
10:837407:A:G       0.30200
11:61619829:C:A     0.01300
12:115890922:T:C    0.04200
12:115891403:C:T    0.00000
12:11962573:T:C     0.47400
12:131045130:G:A    0.08550
12:17444733:A:T     0.25700
12:4400808:C:T      0.17000
12:4405389:A:G      0.20300
12:51045449:A:G     0.05000
12:57540848:C:G     0.26300
12:6412036:T:G      0.04600
12:6421174:A:T      0.21300
13:111075881:C:T    0.12700
14:54419106:A:C     0.04900
14:85114169:G:A     0.01550
15:28365618:A:G     0.00300
15:28410491:C:T     0.26600
15:31741216:C:A     0.13600
15:32994756:T:C     0.00700
15:33001734:T:A     0.22400
15:33010412:G:A     0.23000
15:33010736:G:A     0.23000
15:33012502:A:G     0.31400
15:68177162:G:A     0.18800
16:16993003:A:G     0.50000
16:68812301:A:T     0.42900
16:68820946:G:A     0.39100
16:86339315:T:C     0.03800
16:86340448:G:C     0.00200
16:86695720:G:C     0.00100
17:70413253:G:A     0.01900
18:9701782:C:T      0.22700
19:41860296:A:G  

In [194]:
df_gt_conflicts[~df_gt_conflicts.patient_id.isin(plate_diff_patients)].groupby(['patient_id', 'ID'], as_index=False).GP_max.apply(lambda s: abs(s.iloc[0]-s.iloc[1])).groupby('ID').GP_max.mean().round(1).value_counts().sort_index()

0.0    60
0.1    68
0.2    41
0.3    26
0.4    16
0.5    10
Name: GP_max, dtype: int64

In [211]:
def diff_sample_types(df):
    return df[df.sample_type=='10'].GP_max.iloc[0] - df[df.sample_type=='11'].GP_max.iloc[0]

df_gt_conflicts[~df_gt_conflicts.patient_id.isin(plate_diff_patients)].groupby(['patient_id', 'ID']).apply(diff_sample_types).round(1).value_counts().sort_index()

-0.5      8
-0.4     10
-0.3     14
-0.2     16
-0.1     39
 0.0    138
 0.1     46
 0.2     21
 0.3     37
 0.4     26
 0.5     10
dtype: int64

In [103]:
# df_argmax_conflicts
len(df_gt_conflicts[df_gt_conflicts.GT!=df_gt_conflicts.GP.apply(lambda a: a.index(max(a)))])/len(df_gt_conflicts)

0.012711864406779662

In [ ]:
len(df_gt_conflicts)

### Removing Outliers SNPs

In [220]:
df_t

,IS_IMPUTED,AF,MAF,R2,ER2,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,sample_id,gt_scores,patient_id,GT,GP,GD,HDS,sample_type
0,TYPED,0.08269,0.08269,0.98341,0.74366,1,14665131,1:14665131:C:T,C,T,.,PASS,AF=0.08269;MAF=0.08269;R2=0.98341;ER2=0.74366;...,GT:DS:HDS:GP,TCGA-AA-3693-10A-01D-0903-01,"1|1:2:1,1:0,0,1",TCGA-AA-3693,2,"[0, 0, 1]",2.000,"[1, 1]",10
1,IMPUTED,0.09059,0.09059,0.96957,NaN,1,18130969,1:18130969:G:C,G,C,.,PASS,AF=0.09059;MAF=0.09059;R2=0.96957;IMPUTED,GT:DS:HDS:GP,TCGA-AA-3693-10A-01D-0903-01,"1|0:0.997:0.997,0:0.003,0.997,0",TCGA-AA-3693,1,"[0.003, 0.997, 0]",0.997,"[0.997, 0]",10
2,IMPUTED,0.07441,0.07441,0.95783,NaN,1,22587728,1:22587728:T:C,T,C,.,PASS,AF=0.07441;MAF=0.07441;R2=0.95783;IMPUTED,GT:DS:HDS:GP,TCGA-AA-3693-10A-01D-0903-01,"0|0:0:0,0:1,0,0",TCGA-AA-3693,0,"[1, 0, 0]",0.000,"[0, 0]",10
3,IMPUTED,0.02371,0.02371,0.73719,NaN,1,31449766,1:31449766:A:G,A,G,.,PASS,AF=0.02371;MAF=0.02371;R2=0.73719;IMPUTED,GT:DS:HDS:GP,TCGA-AA-3693-10A-01D-0903-01,"0|1:0.6:0,0.6:0.4,0.6,0",TCGA-AA-3693,1,"[0.4, 0.6, 0]",0.600,"[0, 0.6]",10
4,IMPUTED,0.47909,0.47909,0.89480,NaN,1,38455891,1:38455891:G:C,G,C,.,PASS,AF=0.47909;MAF=0.47909;R2=0.8948;IMPUTED,GT:DS:HDS:GP,TCGA-AA-3693-10A-01D-0903-01,"1|0:0.962:0.962,0:0.038,0.962,0",TCGA-AA-3693,1,"[0.038, 0.962, 0]",0.962,"[0.962, 0]",10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305365,TYPED,0.71606,0.28394,0.99778,0.97242,22,29130012,22:29130012:T:C,T,C,.,PASS,AF=0.71606;MAF=0.28394;R2=0.99778;ER2=0.97242;...,GT:DS:HDS:GP,TCGA-F4-6461-10A-01D-1770-01,"0|1:1:0,1:0,1,0",TCGA-F4-6461,1,"[0, 1, 0]",1.000,"[0, 1]",10
305366,TYPED,0.39010,0.39010,0.99637,0.96381,22,29192670,22:29192670:T:C,T,C,.,PASS,AF=0.3901;MAF=0.3901;R2=0.99637;ER2=0.96381;TYPED,GT:DS:HDS:GP,TCGA-F4-6461-10A-01D-1770-01,"0|1:1:0,1:0,1,0",TCGA-F4-6461,1,"[0, 1, 0]",1.000,"[0, 1]",10
305367,IMPUTED,0.58853,0.41147,0.97866,NaN,22,31139318,22:31139318:G:A,G,A,.,PASS,AF=0.58853;MAF=0.41147;R2=0.97866;IMPUTED,GT:DS:HDS:GP,TCGA-F4-6461-10A-01D-1770-01,"1|0:0.999:0.999,0:0.001,0.999,0",TCGA-F4-6461,1,"[0.001, 0.999, 0]",0.999,"[0.999, 0]",10
305368,TYPED,0.22413,0.22413,0.98753,0.87478,22,31838518,22:31838518:C:T,C,T,.,PASS,AF=0.22413;MAF=0.22413;R2=0.98753;ER2=0.87478;...,GT:DS:HDS:GP,TCGA-F4-6461-10A-01D-1770-01,"0|1:1:0,1:0,1,0",TCGA-F4-6461,1,"[0, 1, 0]",1.000,"[0, 1]",10


### Resolving Sample Conflicts

In [215]:
df_gt_conflicts.patient_id.nunique()

43

In [ ]:
df_t.

## Variants Entropy 

In [159]:
df_t.GP.apply(lambda e: entropy(e)).round(1).value_counts().sort_index()

0.0    243660
0.1     30641
0.2     11811
0.3      5778
0.4      3445
0.5      2525
0.6      2505
0.7      2801
0.8       554
0.9       268
1.0       212
Name: GP, dtype: int64

In [214]:
df_t.patient_id.nunique()

459

In [212]:
df_t.groupby('sample_type').patient_id.nunique()

sample_type
10    419
11     92
Name: patient_id, dtype: int64

1652

## GD

In [37]:
# np.unique(df_t.GD.values.round(1)#.value_counts()
df_t.groupby(df_t.GD.values.round(1)).agg({
    'sample_id': 'count',
    'GT': 'mean'
})

,sample_id,GT
0.0,148093,0.000000
0.1,2146,0.000000
0.2,1041,0.000000
0.3,693,0.000000
0.4,599,0.000000
0.5,570,0.438596
0.6,631,0.931854
0.7,716,0.958101
0.8,1091,0.984418
0.9,4221,0.998815


## HDS

In [38]:
df_t.HDS

0             [1, 1]
1         [0.997, 0]
2             [0, 0]
3           [0, 0.6]
4         [0.962, 0]
             ...    
305365        [0, 1]
305366        [0, 1]
305367    [0.999, 0]
305368        [0, 1]
305369    [0, 0.922]
Name: HDS, Length: 305370, dtype: object

# Data Saving

## Ex0 - Resolving Conflicts by avg Entropy

In [8]:
entropy_by_sample = df_t.groupby(['sample_id', 'patient_id'], as_index=False).GP.apply(lambda s: s.apply(lambda e: entropy(e)).mean())
entropy_by_sample.head()

,sample_id,patient_id,GP
0,TCGA-3L-AA1B-10A-01D-A36Z-01,TCGA-3L-AA1B,0.052868
1,TCGA-4N-A93T-10A-01D-A36Z-01,TCGA-4N-A93T,0.056384
2,TCGA-4T-AA8H-10B-01D-A40O-01,TCGA-4T-AA8H,0.059962
3,TCGA-5M-AAT4-10A-01D-A40O-01,TCGA-5M-AAT4,0.052557
4,TCGA-5M-AAT5-10A-01D-A40O-01,TCGA-5M-AAT5,0.055139


In [ ]:
entropy_by_sample.rename(columns={"GP": "avg_entropy"}, inplace=True)

In [13]:
sample_per_patient = entropy_by_sample.groupby('patient_id').apply(lambda d: d.sample_id.iloc[d.avg_entropy.argmin()])
sample_per_patient.head()

patient_id
TCGA-3L-AA1B    TCGA-3L-AA1B-10A-01D-A36Z-01
TCGA-4N-A93T    TCGA-4N-A93T-10A-01D-A36Z-01
TCGA-4T-AA8H    TCGA-4T-AA8H-10B-01D-A40O-01
TCGA-5M-AAT4    TCGA-5M-AAT4-10A-01D-A40O-01
TCGA-5M-AAT5    TCGA-5M-AAT5-10A-01D-A40O-01
dtype: object

In [31]:
label_per_patient = {}
for sample_id in sample_per_patient:
    label_per_patient[sample_id] = list(df[sample_id].apply(lambda elem: int(elem[0]) + int(elem[2])))
df_0 = pd.DataFrame(label_per_patient.items(), columns=['sample_id', 'GT_array'])
df_0['patient_id'] = df_0.sample_id.str[:12]
display(df_0.head(1))
display(df_0.shape)

,sample_id,GT_array,patient_id
0,TCGA-3L-AA1B-10A-01D-A36Z-01,"[0, 0, 0, 0, 1, 1, 2, 0, 0, 0, 0, 1, 0, 1, 1, ...",TCGA-3L-AA1B


(459, 3)

In [32]:
df_0.to_csv('variant_labels_0.csv', index=False)